In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

d = { "none" : 1,
"del_h1" : 2,
"del_h2" : 3,
"del_hom" : 4,
"dup_h1" : 5,
"dup_h2" : 6,
"dup_hom" : 7,
"inv_h1" : 8,
"inv_h2" : 9,
"inv_hom" : 10,
"idup_h1" : 11,
"idup_h2" : 12,
"complex" : 13}

colors = {"none" : "#F8F8F8",
"del_h1" : "#77AADD",
"del_h2" : "#4477AA",
"del_hom" : "#114477",
"dup_h1" : "#CC99BB",
"dup_h2" : "#AA4488",
"dup_hom" : "#771155",
"inv_h1" : "#DDDD77",
"inv_h2" : "#AAAA44",
"inv_hom" : "#777711",
"idup_h1" : "#DDAA77",
"idup_h2" : "#AA7744",
"complex" : "#774411"}

df = pd.read_csv("/g/korbel2/weber/workspace/R_dev/stringent_filterTRUE.tsv", sep="\t")
df = df.loc[df["chrom"] == "chr13"]
df["ID"] = df["chrom"] + "_" + df["start"].astype(str) + "_" + df["end"].astype(str)
df


binbed = pd.read_csv("/g/korbel2/weber/workspace/R_dev/bin_200kb_all.bed", sep="\t", names=["chrom", "start", "end", "bin_id"])
binbed["ID"] = binbed["chrom"] + "_" + binbed["start"].astype(str) + "_" + binbed["end"].astype(str)
binbed["chrom"] = pd.Categorical(binbed["chrom"], categories=["chr{}".format(e) for e in range(1,23)] + ["chrX", "chrY"], ordered=True)
binbed = binbed.sort_values(by=["chrom", "start", "end"]).reset_index(drop=True)
binbed = binbed.loc[~binbed["chrom"].isin(["chrY"])]


l = list()

def process_row(r):
    # print(r["cell"])
    tmp_r = binbed.loc[(binbed["chrom"] == r["chrom"]) & (binbed["start"] >= r["start"]) & (binbed["end"] <= r["end"])]
    tmp_r["cell"] = r["cell"]
    tmp_r["sv_call_name"] = r["sv_call_name"]
    tmp_r["af"] = r["af"]
    tmp_r["llr_to_ref"] = r["llr_to_ref"]
    # print(tmp_r)
    l.append(tmp_r)

def process_sv(tmp_df):
    tmp_df.apply(lambda r: process_row(r), axis=1)
    # print(tmp_df)
    # exit()


df.groupby("cell").apply(lambda r: process_sv(r))
# print(l)
processed_df = pd.concat(l)
processed_df["ID"] = processed_df["chrom"].astype(str) + "_" + processed_df["start"].astype(str) + "_" + processed_df["end"].astype(str)


binbed_not_used = binbed.loc[~binbed["ID"].isin(processed_df.ID.unique().tolist())]
concat_df = pd.concat([processed_df, binbed_not_used])

concat_df.loc[concat_df["llr_to_ref"] == np.inf, "llr_to_ref"] = concat_df.loc[concat_df["llr_to_ref"] != np.inf]["llr_to_ref"].max()
concat_df["start"] = concat_df["start"].astype(int)
concat_df.loc[concat_df["chrom"] == "chr13"].sort_values(by=["start"]).groupby(["start"])["cell"].apply(list).reset_index().sort_values(by="start")



# pivot_concat_df = concat_df.pivot(index="ID", values="llr_to_ref", columns="cell")
# tmp = pivot_concat_df.reset_index().ID.str.split("_", expand=True)
# tmp.columns = ["chrom", "start", "end"]
# tmp["start"] = tmp["start"].astype(int)
# tmp["end"] = tmp["end"].astype(int)

# pivot_concat_df = pd.concat([pivot_concat_df.reset_index(), tmp], axis=1).drop(pivot_concat_df.columns[0], axis=1).sort_values(by=["chrom", "start", "end"]).reset_index(drop=True)
# pivot_concat_df

,start,cell
0,0,[nan]
1,200000,[nan]
2,400000,[nan]
3,600000,[nan]
4,800000,[nan]
...,...,...
567,113400000,"[BM510x3PE20411, BM510x3PE20414, BM510x3PE2041..."
568,113600000,"[BM510x3PE20403, BM510x3PE20414, BM510x3PE2040..."
569,113800000,"[BM510x3PE20425, BM510x3PE20415, BM510x3PE2041..."
570,114000000,"[BM510x3PE20426, BM510x3PE20423, BM510x3PE2042..."


In [2]:
df

,chrom,start,end,sample,cell,class,scalar,num_bins,sv_call_name,sv_call_haplotype,sv_call_name_2nd,sv_call_haplotype_2nd,llr_to_ref,llr_to_2nd,af,ID
1052,chr13,16200000,17400000,RPE1_Mix,BM510x3PE20403,WW,1,6,inv_h1,110,ref_hom,1010,7.685908,7.685908e+00,0.27,chr13_16200000_17400000
1053,chr13,16200000,17400000,RPE1_Mix,BM510x3PE20410,CC,1,6,inv_h1,110,inv_hom,101,29.424787,3.781545e+00,0.27,chr13_16200000_17400000
1054,chr13,16200000,17400000,RPE1_Mix,BM510x3PE20411,WW,1,6,inv_hom,101,inv_h1,110,84.139636,4.796717e+01,0.11,chr13_16200000_17400000
1055,chr13,16200000,17400000,RPE1_Mix,BM510x3PE20415,WC,1,6,inv_hom,101,ref_hom,1010,11.479274,1.147927e+01,0.11,chr13_16200000_17400000
1056,chr13,16200000,17400000,RPE1_Mix,BM510x3PE20417,CC,1,6,inv_hom,101,inv_h2,1001,30.195149,7.671533e+00,0.11,chr13_16200000_17400000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1299,chr13,107800000,114364328,RPE1_Mix,BM510x3PE20422,WW,1,33,del_h1,10,del_h2,1000,16.538669,8.881784e-16,0.19,chr13_107800000_114364328
1300,chr13,107800000,114364328,RPE1_Mix,BM510x3PE20423,WW,1,33,del_h1,10,del_h2,1000,16.538669,8.881784e-16,0.19,chr13_107800000_114364328
1301,chr13,107800000,114364328,RPE1_Mix,BM510x3PE20424,CC,1,33,del_h1,10,del_h2,1000,16.538669,8.881784e-16,0.19,chr13_107800000_114364328
1302,chr13,107800000,114364328,RPE1_Mix,BM510x3PE20425,WW,1,33,del_h1,10,del_h2,1000,16.538669,8.881784e-16,0.19,chr13_107800000_114364328


In [3]:
!pip install floweaver ipysankeywidget

In [4]:
!jupyter nbextension enable --py --sys-prefix ipysankeywidget

Config option `kernel_spec_manager_class` not recognized by `EnableNBExtensionApp`.
Enabling notebook extension jupyter-sankey-widget/extension...
      - Validating: OK


In [5]:
import pandas as pd
flows = pd.read_csv('simple_fruit_sales.csv')
flows

,source,target,type,value
0,farm1,Mary,apples,5
1,farm1,James,apples,3
2,farm2,Fred,apples,10
3,farm2,Fred,bananas,10
4,farm2,Susan,bananas,5
5,farm3,Susan,apples,10
6,farm4,Susan,bananas,1
7,farm5,Susan,bananas,1
8,farm6,Susan,bananas,1


In [6]:
from ipysankeywidget import SankeyWidget
SankeyWidget(links=flows.to_dict('records'))

SankeyWidget(links=[{'source': 'farm1', 'target': 'Mary', 'type': 'apples', 'value': 5}, {'source': 'farm1', '…

In [7]:
concat_df.loc[concat_df["chrom"] == "chr13"].groupby("ID")["cell"].count().sort_index()

ID
chr13_0_200000                0
chr13_100000000_100200000    22
chr13_10000000_10200000       0
chr13_1000000_1200000         0
chr13_100200000_100400000    22
                             ..
chr13_99000000_99200000      22
chr13_99200000_99400000      22
chr13_99400000_99600000      22
chr13_99600000_99800000      22
chr13_99800000_100000000     22
Name: cell, Length: 572, dtype: int64

In [8]:
pd.pivot_table(concat_df.loc[concat_df["chrom"] == "chr13"].groupby(["ID", "sv_call_name"])["cell"].count().reset_index(), index="ID", columns="sv_call_name", values="cell").fillna(0)

sv_call_name,del_h1,del_h2,idup_h2,inv_h1,inv_h2,inv_hom
ID,,,,,,
chr13_100000000_100200000,1.0,20.0,0.0,0.0,0.0,1.0
chr13_100200000_100400000,1.0,20.0,0.0,0.0,0.0,1.0
chr13_100400000_100600000,1.0,20.0,0.0,0.0,0.0,1.0
chr13_100600000_100800000,1.0,20.0,0.0,0.0,0.0,1.0
chr13_100800000_101000000,1.0,20.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...
chr13_99000000_99200000,1.0,20.0,0.0,0.0,0.0,1.0
chr13_99200000_99400000,1.0,20.0,0.0,0.0,0.0,1.0
chr13_99400000_99600000,1.0,20.0,0.0,0.0,0.0,1.0


In [24]:
test = pd.merge(
    concat_df.loc[concat_df["chrom"] == "chr13", ["chrom", "start", "end", "ID"]].sort_values(by="start").drop_duplicates(),
    pd.pivot_table(concat_df.loc[concat_df["chrom"] == "chr13"].groupby(["ID", "sv_call_name"])["cell"].count().reset_index(), index="ID", columns="sv_call_name", values="cell").fillna(0).reset_index(),
    on=["ID"], how="left").sort_values(by="start")
test = test.melt(id_vars=["start", "end"], value_vars=["del_h1", "del_h2", "idup_h2", "inv_h1", "inv_h2", "inv_hom"]).rename({"start" : "source", "end": "target", "variable": "type"}, axis=1)
test["value"] = test["value"].fillna(0)

test
# source	target	type	value

,source,target,type,value
0,0,200000,del_h1,0.0
1,200000,400000,del_h1,0.0
2,400000,600000,del_h1,0.0
3,600000,800000,del_h1,0.0
4,800000,1000000,del_h1,0.0
...,...,...,...,...
3427,113400000,113600000,inv_hom,0.0
3428,113600000,113800000,inv_hom,0.0
3429,113800000,114000000,inv_hom,0.0
3430,114000000,114200000,inv_hom,0.0


In [29]:
from ipysankeywidget import SankeyWidget
SankeyWidget(links=test.head(3000).tail(100).to_dict('records'))

SankeyWidget(links=[{'source': 8000000, 'target': 8200000, 'type': 'inv_hom', 'value': 0.0}, {'source': 820000…

In [30]:
!pip install d3blocks

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.0/587.0 kB 12.6 MB/s eta 0:00:00m eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.6/240.6 kB 2.7 MB/s eta 0:00:000:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 55.8 MB/s eta 0:00:000m eta 0:00:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.4/99.4 kB 1.4 MB/s eta 0:00:000 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.6/204.6 kB 5.3 MB/s eta 0:00:00 MB/s eta 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 22.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-louvain: filename=python_louvain-0.16-py3-none-any.whl size=9388 sha256=d5261262c8484bffdacfc152579d2c9753e2bf90be51512bad4dcf6a9958a47e
  Stored in directory: /home/tweber/.cache/pip/wheels/d0/b0/d7/6dd26c3817810fa379088eaeb755a01d9a2a411c37632079d1
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=089697